<a href="https://colab.research.google.com/github/Avi-nashkumar/Smaple_Project1/blob/master/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=ea0f8684b40684249733b4691cfdf6dc614ea9aeb6f2cdec039e1963ddbf3d08
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=c018c8575cb4057b9eaa02f509d0d8e261b4a2db869f73d3da865587b50b5721
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [5]:
fashion_mnist= keras.datasets.fashion_mnist

In [6]:
(train_images,train_label),(test_images,test_label)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
train_images= train_images/255.0
test_images= test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)


In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 24s]
val_accuracy: 0.903333306312561

Best val_accuracy So Far: 0.9125000238418579
Total elapsed time: 00h 01m 56s
INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                3539040   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,605,706
Trainable params: 3,605,706
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_label, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1121 - accuracy: 0.9591 - val_loss: 0.2961 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0727 - accuracy: 0.9719 - val_loss: 0.3155 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0460 - accuracy: 0.9836 - val_loss: 0.3774 - val_accuracy: 0.9115
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0304 - accuracy: 0.9892 - val_loss: 0.4241 - val_accuracy: 0.9095
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0256 - accuracy: 0.9913 - val_loss: 0.4981 - val_accuracy: 0.9112
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0182 - accuracy: 0.9936 - val_loss: 0.5379 - val_accuracy: 0.9107
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0176 - accuracy: 0.9937 - val_loss: 0.5971 - val_accuracy